# Transformer






In [2]:
import torch
checkpoints_path = "/root/eigenestimation/outputs/eigenmodels/"

eigenmodel_path = checkpoints_path + "tinystories-8M-eigenmodel.pt"
eigenmodel = torch.load(eigenmodel_path)['model']
from eigenestimation.evaluation.top_logits import compute_jacobian
from importlib import reload
# reload compute jacobian
import sys


In [3]:
import sys
sys.path.append('/root/eigenestimation')  # Ensure the module path is in sys.path

import eigenestimation.evaluation.top_logits  # Import the full module
from eigenestimation.evaluation.top_logits import compute_jacobian  # Import the function

from importlib import reload
reload(eigenestimation.evaluation.top_logits)  # Reload the entire module

# If you need to ensure the function is updated, re-import it
from eigenestimation.evaluation.top_logits import compute_jacobian

In [4]:
X_data_path = checkpoints_path + "tinystories-8M-X_data.pt"
attributions_path = checkpoints_path + "tinystories-8M-circuit_attributions.pt"
eigenmodel_path = checkpoints_path + "tinystories-8M-eigenmodel.pt"
X_data = torch.load(X_data_path)
attributions = torch.load(attributions_path)
eigenmodel = torch.load(eigenmodel_path)['model']
tokenizer = eigenmodel.model.tokenizer
frac_activated = torch.load(eigenmodel_path)['frac_activated']

In [108]:
import os
import subprocess
import re

def sanitize_for_latex(text):
    """Sanitize text for LaTeX by escaping special characters and removing invalid Unicode."""
    text = text.encode('utf-8', 'ignore').decode('utf-8')  # Remove invalid Unicode
    text = text.replace("�", "")  # Explicitly remove replacement character
    text = re.sub(r'([%_&#$^{}])', r'\\\1', text)  # Escape LaTeX special characters
    text = text.replace('\n', ' ')  # Replace newlines with a space for table formatting
    text = text.strip()  # Remove unnecessary whitespace
    return text

def generate_latex_from_examples(attributions, X_data, tokenizer, circuit_idxs, output_filename="circuit_examples"):
    """
    Generates a LaTeX file from transformer attributions and compiles it into a PDF.
    """
    latex_content = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}  % Ensure UTF-8 encoding
\usepackage{booktabs}
\usepackage{array}
\usepackage[a4paper, margin=1in]{geometry} % Better margins
\usepackage{adjustbox} % For text fitting
\usepackage{multirow} % Allows merging cells
\begin{document}

\section*{Circuit Analysis Examples}

\noindent Below is a unified table of all circuits.

\begin{adjustbox}{max width=\textwidth}
\small
\begin{tabular}{p{0.1\textwidth} p{0.5\textwidth} p{0.4\textwidth}}
\toprule
\textbf{Id ($P_{act}$)} & \textbf{Input Text} & \textbf{Top Logits} \\
\midrule
"""

    # Iterate through attributions and format them in LaTeX
    for circuit_idx in circuit_idxs:
        activation = attributions[circuit_idx]['activation']
        circuit_label = f"{circuit_idx} ({activation:.3f})"
        first_row = True  # Flag to ensure multirow cell only appears once

        examples = attributions[circuit_idx]['top_examples']
        num_rows = len(examples)  # Get number of rows for multirow

        for i, example in enumerate(examples):
            tokens = X_data[example['sample_id']][:example['token_id']+1]
            text = sanitize_for_latex(tokenizer.decode(tokens.long()))
            jac = compute_jacobian(eigenmodel, tokens.to('cuda'), circuit_idx, device='cuda')
            top_token_idxs = (jac).argsort(descending=True)[:5]
            top_logits = sanitize_for_latex(', '.join([eigenmodel.model.tokenizer.decode(token_idx) for token_idx in top_token_idxs]))

            # Multi-row cell for Circuit Index (P Activation) column
            if first_row:
                latex_content += f"\\multirow{{{num_rows}}}{{*}}{{\\textbf{{{circuit_label}}}}} & {text} & {top_logits} \\\\\n"
                first_row = False
            else:
                latex_content += f" & {text} & {top_logits} \\\\\n"

        latex_content += "\\midrule\n"  # Add a horizontal line after each circuit section

    latex_content += r"""
\bottomrule
\end{tabular}
\end{adjustbox}

\end{document}
"""

    # Save LaTeX content to a file
    tex_filename = f"{output_filename}.tex"
    with open(tex_filename, "w", encoding="utf-8") as f:
        f.write(latex_content)

    print(f"LaTeX file '{tex_filename}' created successfully.")

    # Compile the LaTeX file into a PDF
    try:
        subprocess.run(["pdflatex", "-interaction=nonstopmode", tex_filename], check=True)
        print(f"PDF '{output_filename}.pdf' generated successfully.")

        # Cleanup auxiliary files
        for ext in [".aux", ".log"]:
            aux_file = f"{output_filename}{ext}"
            if os.path.exists(aux_file):
                os.remove(aux_file)

    except subprocess.CalledProcessError:
        print("Error: Failed to compile the LaTeX file.")


LaTeX file 'circuit_examples.tex' created successfully.
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./circuit_examples.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/booktabs/booktabs.sty)
(/usr/share/texlive/texmf-dist/tex/latex/tools/array.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/adjustbox/adjustbox.sty
(/usr/share/t

In [110]:
circuit_idxs = [69, 4, 3, 2, 5, 7, 9, 11, 16, 17, 18, 24, 26, 29, 31, 33, 34, 44, 46, 47, 51, 54, 62, 63, 74, 79, 86, 91, 98, 99]
generate_latex_from_examples(attributions, X_data, tokenizer, list(range(100)), 'circuit_examples_all')

generate_latex_from_examples(attributions, X_data, tokenizer, list(range(10)), 'circuit_examples_great')


LaTeX file 'circuit_examples_all.tex' created successfully.
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./circuit_examples_all.tex
LaTeX2e <2021-11-15> patch level 1
L3 programming layer <2022-01-21>
(/usr/share/texlive/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX document class
(/usr/share/texlive/texmf-dist/tex/latex/base/size10.clo))
(/usr/share/texlive/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/share/texlive/texmf-dist/tex/latex/booktabs/booktabs.sty)
(/usr/share/texlive/texmf-dist/tex/latex/tools/array.sty)
(/usr/share/texlive/texmf-dist/tex/latex/geometry/geometry.sty
(/usr/share/texlive/texmf-dist/tex/latex/graphics/keyval.sty)
(/usr/share/texlive/texmf-dist/tex/generic/iftex/ifvtex.sty
(/usr/share/texlive/texmf-dist/tex/generic/iftex/iftex.sty)))
(/usr/share/texlive/texmf-dist/tex/latex/adjustbox/adjustbox.sty
(/usr


kpathsea: Running mktexpk --mfmode / --bdpi 600 --mag 1+0/600 --dpi 600 tcrm0900
mktexpk: Running mf-nowin -progname=mf \mode:=ljfour; mag:=1+0/600; nonstopmode; input tcrm0900
This is METAFONT, Version 2.71828182 (TeX Live 2022/dev/Debian) (preloaded base=mf)

(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/tcrm0900.mf
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exbase.mf)
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/tcrm.mf
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/txsymb.mf
 Ok (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/exaccess.mf
 Ok) (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/txpseudo.mf
 Ok) (/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/txaccent.mf
 Ok [0] [1] [2] [3] [4] [5] [6] [7] [8] [9] [10] [11] [12] [27] [29])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/txgen.mf
 Ok [100] [109] [98] [99] [108])
(/usr/share/texlive/texmf-dist/fonts/source/jknappen/ec/txsymbol.mf
 Ok [13] [18] [21]

 )
(see the transcript file for additional information) </root/.texlive2021/texmf-
var/fonts/pk/ljfour/jknappen/ec/tcrm0900.600pk></usr/share/texlive/texmf-dist/f
onts/type1/public/amsfonts/cm/cmbx12.pfb></usr/share/texlive/texmf-dist/fonts/t
ype1/public/amsfonts/cm/cmbx9.pfb></usr/share/texlive/texmf-dist/fonts/type1/pu
blic/amsfonts/cm/cmmi6.pfb></usr/share/texlive/texmf-dist/fonts/type1/public/am
sfonts/cm/cmmi9.pfb></usr/share/texlive/texmf-dist/fonts/type1/public/amsfonts/
cm/cmr10.pfb></usr/share/texlive/texmf-dist/fonts/type1/public/amsfonts/cm/cmr9
.pfb>
Output written on circuit_examples_all.pdf (2 pages, 102253 bytes).
Transcript written on circuit_examples_all.log.
PDF 'circuit_examples_all.pdf' generated successfully.
LaTeX file 'circuit_examples_great.tex' created successfully.
This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
(./circuit_examples_great.tex
LaTeX2e <202